In [140]:
!pip install lightgbm

In [141]:
import pandas as pd
import json
import datetime as dt
import numpy as np
## m

## Load Data

In [142]:
def get_one_hot(df, feature):
    one_hot = pd.get_dummies(df[feature])
    one_hot.columns = [feature+'_'+str(i.lower().translate(str.maketrans('áéíóúü','aeiouu'))) for i in one_hot.columns]
    return one_hot

In [143]:
with open('../data/train.json', mode='r') as f:
    data = json.load(f)
list_dict_data = []
for i in data:
    dict_data = {}
    for j in i:
        dict_data[list(j.keys())[0]] = j[list(j.keys())[0]]
    list_dict_data.append(dict_data)
train = pd.DataFrame(list_dict_data)
train.columns = [i.lower().translate(str.maketrans('áéíóúü','aeiouu')) for i in list(train.columns)]
train['nombre_campania'] = train['nombre_campania'].replace({'GASTRONOMIA':'GASTRONOMY','GASTRONOMÃ�A':'GASTRONOMY'})

train.drop(['moneda', 'country', 'brand'], axis=1, inplace=True)
train['fecha_publicacion_anuncio'] = train['fecha_publicacion_anuncio'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if x else np.nan)
train['fecha_captura_informacion'] = train['fecha_captura_informacion'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if x else np.nan)
train['interacciones'] = train['interacciones'].astype('int32')
train['duracion_anuncio_total'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days

##CREANDO CAMPO DIFF DIAS JLP
train = train.sort_values(['id_anuncio','fecha_captura_informacion'], ascending = True)
grouped = train.groupby('id_anuncio')
diff = lambda x: x['fecha_captura_informacion'].replace(0, np.NaN).diff() 
train["DIFF"] = grouped.apply(diff).reset_index(0, drop = True).dt.days
train['DIFF'].fillna(train['duracion_anuncio_total'], inplace = True)

mask = (train.id_anuncio.astype(str)=='None') & (train.impresiones==0)
train['mask'] = mask.map({True:0,False:1})## train[~mask].copy()

train.drop(['fecha_publicacion_anuncio', 'fecha_captura_informacion'], axis=1, inplace=True) ## OJO
# train.drop(['id_anuncio','id_conjunto_anuncios','id_campania'], axis=1, inplace=True) ## OJO
train = train.set_index('id_observacion')
## aplicar la variable compuesta a id_anuncion y  captura de info
# var_cuanti = ['duracion_anuncio', 'gasto', 'visitas_al_50', 'visitas_al_75', 'visitas_al100','impresiones','DIFF']
var_categ = ['fuente','objetivo_campania','formato','nombre_campania']

list_df = []
for i in var_categ:
    df = get_one_hot(train, i)
    list_df.append(df)
train_categ = pd.concat(list_df,axis=1)

train_final = train.join(train_categ)
train_final.drop(var_categ, axis=1, inplace=True) ### OJO
Y = train_final['interacciones']
train_final.drop(['interacciones'], axis=1, inplace=True)
train_final

,id_anuncio,id_conjunto_anuncios,id_campania,gasto,visitas_al_50,visitas_al_75,visitas_al100,impresiones,duracion_anuncio_total,DIFF,...,formato_video,nombre_campania_aon,nombre_campania_cusquena,nombre_campania_equity,nombre_campania_fin de ano,nombre_campania_gastronomia promo,nombre_campania_gastronomy,nombre_campania_promo cusquena,nombre_campania_terra cusco,nombre_campania_terra lima
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_000129,"1,136,292,264,672,720,000",i25608626,i21206311,437.500000,56939.0,42590.0,34748.0,1.139250e+05,3.0,3.0,...,1,1,0,0,0,0,0,0,0,0
ID_000126,"1,138,211,077,668,450,000",i25666138,i21241415,250.000000,51455.0,42138.0,38094.0,7.094100e+04,0.0,0.0,...,1,1,0,0,0,0,0,0,0,0
ID_000127,"1,138,211,077,668,450,000",i25666138,i21241415,250.000000,51380.0,41780.0,37794.0,7.208100e+04,1.0,1.0,...,1,1,0,0,0,0,0,0,0,0
ID_000128,"1,138,211,077,668,450,000",i25666138,i21241415,250.000000,50290.0,40958.0,36977.0,7.106100e+04,2.0,1.0,...,1,1,0,0,0,0,0,0,0,0
ID_000125,"1,138,211,077,668,450,000",i25666138,i21241415,250.000000,52213.0,42609.0,38686.0,7.240000e+04,3.0,1.0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_005561,None,None,None,461.538461,NaN,NaN,NaN,6.593407e+04,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
ID_005446,None,None,None,272.500000,NaN,NaN,NaN,1.362500e+05,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
ID_005487,None,None,None,1576.923077,NaN,NaN,NaN,1.051282e+06,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0


In [144]:
train_final[train_final.id_anuncio.astype(str)=='None'].isna().sum()/len(train_final[train_final.id_anuncio.astype(str)=='None'])

id_anuncio                           1.000000
id_conjunto_anuncios                 1.000000
id_campania                          1.000000
gasto                                0.000000
visitas_al_50                        0.424864
visitas_al_75                        0.424864
visitas_al100                        0.424864
impresiones                          0.000000
duracion_anuncio_total               1.000000
DIFF                                 1.000000
mask                                 0.000000
fuente_direct                        0.000000
fuente_fb + ig                       0.000000
fuente_facebook                      0.000000
fuente_instagram                     0.000000
fuente_oath                          0.000000
fuente_search                        0.000000
fuente_teads                         0.000000
fuente_twitter                       0.000000
fuente_youtube                       0.000000
objetivo_campania_brand awareness    0.000000
objetivo_campania_conversions     

In [145]:
train_final[train_final.id_anuncio.astype(str)=='None']['impresiones'].value_counts()

0.000000         96
481481.481500    41
185185.185200    35
85.956522        32
128.086957       29
                 ..
38623.000000      1
23277.666670      1
32746.000000      1
49929.000000      1
36437.000000      1
Name: impresiones, Length: 215, dtype: int64

In [146]:
train_final[train_final.id_anuncio.astype(str)=='None']

,id_anuncio,id_conjunto_anuncios,id_campania,gasto,visitas_al_50,visitas_al_75,visitas_al100,impresiones,duracion_anuncio_total,DIFF,...,formato_video,nombre_campania_aon,nombre_campania_cusquena,nombre_campania_equity,nombre_campania_fin de ano,nombre_campania_gastronomia promo,nombre_campania_gastronomy,nombre_campania_promo cusquena,nombre_campania_terra cusco,nombre_campania_terra lima
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_004548,None,None,None,550.000000,NaN,NaN,NaN,1.375000e+06,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
ID_004551,None,None,None,583.333333,NaN,NaN,NaN,1.296296e+05,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
ID_004549,None,None,None,550.000000,NaN,NaN,NaN,1.375000e+06,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
ID_004550,None,None,None,550.000000,NaN,NaN,NaN,1.375000e+06,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
ID_004552,None,None,None,583.333333,NaN,NaN,NaN,1.296296e+05,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_005561,None,None,None,461.538461,NaN,NaN,NaN,6.593407e+04,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
ID_005446,None,None,None,272.500000,NaN,NaN,NaN,1.362500e+05,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
ID_005487,None,None,None,1576.923077,NaN,NaN,NaN,1.051282e+06,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0


In [147]:
### gasto/duraction_parcial
### gasto/duraction_totla
### impresiones/gasto
### visitas_al_50/gasto
# train_final['gasto/duracion_anuncio_total'] = train_final['gasto']/train_final['duracion_anuncio_total']
# train_final['gasto/DIFF'] = train_final['gasto']/train_final['DIFF']
# train_final['visitas_al_50/gasto'] = train_final['visitas_al_50'].replace({np.nan:0})/train_final['gasto']
# train_final['visitas_al_75/gasto'] =  train_final['visitas_al_75'].replace({np.nan:0})/train_final['gasto']
# train_final['visitas_al100/gasto'] =  train_final['visitas_al100'].replace({np.nan:0})/train_final['gasto']
# train_final['visitas_al_50/gasto'] = train_final['visitas_al_50/gasto'].replace({np.nan:0})
# train_final['visitas_al_75/gasto'] = train_final['visitas_al_75/gasto'].replace({np.nan:0})
# train_final['visitas_al100/gasto'] = train_final['visitas_al100/gasto'].replace({np.nan:0})
# train_final['impresiones/gasto'] = train_final['impresiones']/train_final['gasto']
# train_final

In [148]:
# validation = pd.read_csv('../data/Data_Validacion.csv',sep=';')
validation = pd.read_csv('../data/Data_Validacion.csv',sep=';')
index_val = validation.index
validation.columns = [i.lower().translate(str.maketrans('áéíóúü','aeiouu')) for i in list(validation.columns)]
train['nombre_campania'] = train['nombre_campania'].replace({'GASTRONOMIA':'GASTRONOMY','GASTRONOMÃ�A':'GASTRONOMY'})

validation.drop(['moneda', 'country', 'brand'], axis=1, inplace=True)
validation['fecha_publicacion_anuncio'] = validation['fecha_publicacion_anuncio'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if str(x)!='nan' else np.nan)
validation['fecha_captura_informacion'] = validation['fecha_captura_informacion'].apply(lambda x: dt.datetime.strptime(x, '%a, %d %b %Y 00:00:00 GMT') if str(x)!='nan' else np.nan)
validation['duracion_anuncio_total'] = (validation['fecha_captura_informacion'] - validation['fecha_publicacion_anuncio']).dt.days

##CREANDO CAMPO DIFF DIAS JLP
validation = validation.sort_values(['id_anuncio','fecha_captura_informacion'], ascending = True)
grouped = validation.groupby('id_anuncio')
diff = lambda x: x['fecha_captura_informacion'].replace(0, np.NaN).diff() 
validation["DIFF"] = grouped.apply(diff).reset_index(0, drop = True).dt.days
validation['DIFF'].fillna(validation['duracion_anuncio_total'], inplace = True)
##
mask = (validation.id_anuncio.astype(str)=='nan') & (validation.impresiones==0)
validation['mask'] = mask.map({True:0,False:1})## train[~mask].copy()
# train['duracion_anuncio_parcial'] = (train['fecha_captura_informacion'] - train['fecha_publicacion_anuncio']).dt.days
# mask = train['id_anuncio'].astype(str)=='None'
# train.loc[mask, 'id_anuncio'] = train[mask]['id_observacion']
# train['key'] = train['id_anuncio'].astype(str) +'_'+\
#                train['id_conjunto_anuncios'].astype(str) +'_'+\
#                train['id_campania'].astype(str) +'_'+\
#                train['fecha_captura_informacion'].astype(str)
validation.drop(['fecha_publicacion_anuncio', 'fecha_captura_informacion'], axis=1, inplace=True)
validation.drop(['id_anuncio','id_conjunto_anuncios','id_campania'], axis=1, inplace=True)
# validation = train.set_index('id_observacion')
## aplicar la variable compuesta a id_anuncion y  captura de info
# var_cuanti = ['duracion_anuncio_total', 'gasto', 'visitas_al_50', 'visitas_al_75', 'visitas_al100','impresiones']
var_categ = ['fuente','objetivo_campania','formato','nombre_campania']
list_df = []
for i in var_categ:
    df = get_one_hot(validation, i)
    list_df.append(df)
validation_categ = pd.concat(list_df,axis=1)

validation_final = validation.join(validation_categ)
validation_final.drop(var_categ, axis=1, inplace=True)
validation_final = validation_final.set_index('id_observacion')
validation_final['gasto'] = validation_final['gasto'].replace({'40000%':40000})
validation_final['gasto'] = validation_final['gasto'].astype(float)
validation_final

,gasto,visitas_al_50,visitas_al_75,visitas_al100,impresiones,duracion_anuncio_total,DIFF,mask,fuente_direct,fuente_fb + ig,...,nombre_campania_cusquena,nombre_campania_equity,nombre_campania_fin de ano,nombre_campania_gastronomia,nombre_campania_gastronomia promo,nombre_campania_gastronomy,nombre_campania_gastronomia,nombre_campania_promo cusquena,nombre_campania_terra cusco,nombre_campania_terra lima
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_V_001302,49.843780,0.0,0.0,0.0,10250.00000,0.0,0.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001303,243.361200,42211.0,41183.0,40066.0,45521.00000,2.0,2.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001317,210.701380,35514.0,34674.0,33773.0,37989.00000,3.0,1.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001298,75.990030,13436.0,13075.0,12674.0,14713.00000,4.0,1.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_001304,176.074470,28822.0,28091.0,27341.0,31268.00000,5.0,1.0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_V_002376,461.538461,NaN,NaN,NaN,65934.06593,NaN,NaN,1,0,0,...,0,0,0,1,0,0,0,0,0,0
ID_V_002322,272.500000,NaN,NaN,NaN,136250.00000,NaN,NaN,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ID_V_002377,461.538461,NaN,NaN,NaN,65934.06593,NaN,NaN,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [149]:
# validation_final['gasto/duracion_anuncio_total'] = validation_final['gasto']/validation_final['duracion_anuncio_total']
# validation_final['gasto/DIFF'] = validation_final['gasto']/validation_final['DIFF']
# validation_final['visitas_al_50/gasto'] = validation_final['visitas_al_50'].replace({np.nan:0})/validation_final['gasto']
# validation_final['visitas_al_75/gasto'] =  validation_final['visitas_al_75'].replace({np.nan:0})/train_final['gasto']
# validation_final['visitas_al100/gasto'] =  validation_final['visitas_al100'].replace({np.nan:0})/train_final['gasto']
# validation_final['visitas_al_50/gasto'] = validation_final['visitas_al_50/gasto'].replace({np.nan:0})
# validation_final['visitas_al_75/gasto'] = validation_final['visitas_al_75/gasto'].replace({np.nan:0})
# validation_final['visitas_al100/gasto'] = validation_final['visitas_al100/gasto'].replace({np.nan:0})
# validation_final['impresiones/gasto'] = validation_final['impresiones']/train_final['gasto']
# validation_final

In [150]:
# features = list(set(train_final.columns).intersection(set(validation_final.columns)))
features = list(feature_importances.index)
train_final = train_final[features]
validation_final = validation_final[features]

In [151]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(train_final, Y, test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4449, 29), (1113, 29), (4449,), (1113,))

In [152]:
from lightgbm import LGBMRegressor
learner = LGBMRegressor(n_estimators=1000)
learner.fit(x_train, y_train,  early_stopping_rounds=10, eval_metric="mae",
                eval_set=[(x_train, y_train), (x_test, y_test)], verbose=50)

[50]	training's l1: 136.843	training's l2: 289170	valid_1's l1: 148.808	valid_1's l2: 847317
[100]	training's l1: 109.413	training's l2: 169627	valid_1's l1: 141.908	valid_1's l2: 759353


C:\Users\usuario\anaconda3\envs\venv_backus\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\usuario\anaconda3\envs\venv_backus\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(n_estimators=1000)

In [153]:
validation_final['INTERACCIONES'] = learner.predict(validation_final)
validation_final

C:\Users\usuario\AppData\Local\Temp/ipykernel_9536/1123091501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_final['INTERACCIONES'] = learner.predict(validation_final)


,gasto,impresiones,duracion_anuncio_total,visitas_al100,visitas_al_50,DIFF,objetivo_campania_traffic,visitas_al_75,objetivo_campania_engagement,fuente_fb + ig,...,fuente_facebook,objetivo_campania_conversions,formato_carrusel,formato_video,fuente_youtube,nombre_campania_gastronomia promo,formato_evento,fuente_teads,nombre_campania_terra lima,INTERACCIONES
id_observacion,,,,,,,,,,,,,,,,,,,,,
ID_V_001302,49.843780,10250.00000,0.0,0.0,0.0,0.0,0,0.0,0,0,...,0,0,0,1,0,0,0,1,0,179.106357
ID_V_001303,243.361200,45521.00000,2.0,40066.0,42211.0,2.0,0,41183.0,0,0,...,0,0,0,1,0,0,0,1,0,485.921080
ID_V_001317,210.701380,37989.00000,3.0,33773.0,35514.0,1.0,0,34674.0,0,0,...,0,0,0,1,0,0,0,1,0,181.827236
ID_V_001298,75.990030,14713.00000,4.0,12674.0,13436.0,1.0,0,13075.0,0,0,...,0,0,0,1,0,0,0,1,0,83.192243
ID_V_001304,176.074470,31268.00000,5.0,27341.0,28822.0,1.0,0,28091.0,0,0,...,0,0,0,1,0,0,0,1,0,-90.588186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID_V_002376,461.538461,65934.06593,NaN,NaN,NaN,NaN,0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,8.384599
ID_V_002322,272.500000,136250.00000,NaN,NaN,NaN,NaN,0,NaN,0,0,...,0,0,0,0,1,0,0,0,0,10.331083
ID_V_002377,461.538461,65934.06593,NaN,NaN,NaN,NaN,0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,8.384599


In [154]:
validation_final = validation_final.reset_index()
validation_final = validation_final[['id_observacion','INTERACCIONES']]
# ID_OBSERVACION, INTERACCIONES.
validation_final.columns = ['ID_OBSERVACION', 'INTERACCIONES']
validation_final

,ID_OBSERVACION,INTERACCIONES
0,ID_V_001302,179.106357
1,ID_V_001303,485.921080
2,ID_V_001317,181.827236
3,ID_V_001298,83.192243
4,ID_V_001304,-90.588186
...,...,...
2373,ID_V_002376,8.384599
2374,ID_V_002322,10.331083
2375,ID_V_002377,8.384599
2376,ID_V_002378,8.384599


In [155]:
mask = validation_final['INTERACCIONES']<=0
validation_final.loc[mask,'INTERACCIONES'] = 0

In [156]:
validation_final.to_csv('../results/test6.csv', index=False)

In [157]:
feature_importances = pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=train_final.columns).sort_values(ascending=False)
# feature_importances = feature_importances[feature_importances>0]
# feature_importances.index
feature_importances

gasto                                0.307483
impresiones                          0.204422
duracion_anuncio_total               0.110204
visitas_al100                        0.085714
visitas_al_50                        0.042857
nombre_campania_gastronomy           0.024830
objetivo_campania_traffic            0.024490
DIFF                                 0.023129
visitas_al_75                        0.022109
objetivo_campania_brand awareness    0.017687
objetivo_campania_video views        0.016667
fuente_fb + ig                       0.016327
formato_imagen                       0.014966
objetivo_campania_engagement         0.013605
nombre_campania_aon                  0.012925
fuente_twitter                       0.012245
formato_carrusel                     0.009184
objetivo_campania_reach              0.007823
fuente_facebook                      0.007483
fuente_direct                        0.007143
nombre_campania_promo cusquena       0.007143
fuente_oath                       